In [39]:
# Load the SQL extension
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [2]:
import boto3
import boto3
import time
import json
import time
from dotenv import load_dotenv
from datetime import datetime
import os
from delta.tables import DeltaTable
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, date_add

# Load environment variables from .env file
load_dotenv()

# Initialize AWS credentials from the .env file
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
AWS_SESSION_TOKEN = os.getenv('AWS_SESSION_TOKEN')
AWS_REGION = os.getenv('AWS_REGION')
ROUTIFIC_TOKEN = os.getenv('ROUTIFIC_TOKEN')

In [3]:
# Initialize boto3 client for S3 with your credentials
s3_client = boto3.client(
    's3',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN,
    region_name=AWS_REGION 
)

athena_client = boto3.client(
    'athena',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN,
    region_name=AWS_REGION 
)

In [4]:
# Path to your local JAR files
local_jars = "/Users/borja/Documents/Somniumrema/projects/de/route_optimizer/jars/aws-java-sdk-kinesis-1.12.364.jar"

# Initialize Spark session with Delta and S3 settings
spark = SparkSession.builder \
    .appName("Convert-json-to-parquet") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0,org.apache.hadoop:hadoop-aws:3.3.2,com.amazonaws:aws-java-sdk-bundle:1.11.1026") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.access.key", AWS_ACCESS_KEY_ID) \
    .config("spark.hadoop.fs.s3a.secret.key", AWS_SECRET_ACCESS_KEY) \
    .config("spark.hadoop.fs.s3a.session.token", AWS_SESSION_TOKEN) \
    .config("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com") \
    .config("spark.sql.files.maxPartitionBytes", "134217728") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.executor.memoryOverhead", "4g") \
    .config("spark.driver.memory", "16g") \
    .config("spark.executor.memory", "12g") \
    .config("spark.executor.instances", "16") \
    .config("spark.executor.cores", "16") \
    .config("spark.sql.shuffle.partitions", "500") \
    .config("spark.hadoop.fs.s3a.connection.timeout", "5000") \
    .config("spark.hadoop.fs.s3a.attempts.maximum", "20") \
    .config("spark.hadoop.fs.s3a.connection.maximum", "500") \
    .config("spark.hadoop.fs.s3a.metrics.reporting", "false") \
    .config("spark.sql.broadcastTimeout", "600") \
    .config("spark.sql.autoBroadcastJoinThreshold", "100MB") \
    .config("spark.sql.shuffle.partitions", "500") \
    .config("spark.sql.debug.maxToStringFields", "1000") \
    .config("spark.hadoop.fs.s3a.multipart.size", "104857600") \
    .config("spark.hadoop.fs.s3a.fast.upload", "true") \
    .config("spark.hadoop.fs.s3a.fast.upload.buffer", "disk") \
    .config("spark.hadoop.fs.s3a.fast.upload.active.blocks", "4") \
    .config("spark.hadoop.fs.s3a.fast.upload.active.blocks.threshold", "4096") \
    .config("spark.hadoop.fs.s3a.fast.upload.buffer.size", "2048576") \
    .getOrCreate()

# Optional: Adjust logging level
spark.sparkContext.setLogLevel("WARN")

:: loading settings :: url = jar:file:/Users/borja/Library/Caches/pypoetry/virtualenvs/route-optimizer-AqO2e-Ud-py3.11/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/borja/.ivy2/cache
The jars for the packages stored in: /Users/borja/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-75ade3d5-5287-4f32-9cb5-22ef4e9f54b1;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.hadoop#hadoop-aws;3.3.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.1026 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 175ms :: artifacts dl 12ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.1026 from central in [default]
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-r

In [ ]:
# Adjust the path to your S3 bucket
input_path = "s3a://dispatched-orders/optimized-dispatch-raw/"

# Define the input and output paths in S3
output_path = "s3a://dispatched-orders/optimized-dispatch-bronze/"

from pyspark.sql.functions import to_date, date_add, date_format
from pyspark import StorageLevel

# Read the JSON file with multiLine option enabled
df = (spark.read
      .option("multiLine", "true")
      .json(input_path))

# Dynamic repartitioning based on data size and cluster resources
ideal_partitions = max(df.rdd.getNumPartitions(), 500)  # Adjust minimum to 500 partitions

# Transform the data, repartition, and persist in memory and disk
df_transformed = (df.withColumn("dispatched_date", date_add(to_date(df["finished_at"]), 1))  # Add 1 day for 'dispatched_date'
                  .repartition(ideal_partitions)  # Repartition based on data size
                  .persist(StorageLevel.MEMORY_AND_DISK))  # Persist in memory and disk

# Write the transformed data to the 'bronze' layer, partitioning by 'dispatched_date'
df_transformed.write \
    .mode('overwrite') \
    .partitionBy("dispatched_date") \
    .parquet(output_path)

# Release resources after writing
df_transformed.unpersist()

In [5]:
from pyspark.sql.functions import to_date, date_add, date_format
from pyspark import StorageLevel

In [7]:
# Definir las rutas de entrada y salida
bronze_parquet_path = "s3a://dispatched-orders/optimized-dispatch-bronze/"
silver_parquet_path = "s3a://dispatched-orders/optimized-dispatched-silver/"

In [ ]:

# Definir las rutas de entrada y salida
bronze_parquet_path = "s3a://dispatched-orders/optimized-dispatch-bronze/"
silver_parquet_path = "s3a://dispatched-orders/optimized-dispatched-silver/"

# Leer el archivo Parquet de la capa 'bronze' de S3 y realizar las transformaciones necesarias
# Reparticionamos para reducir el tamaño de las particiones y evitar sobrecarga de memoria
silver_df = (spark.read.parquet(bronze_parquet_path)
              .repartition(500)  # Reparticionar para manejar las operaciones de transformación de manera más eficiente
              .select("_id", "finished_at", "fleet", "id", "input", "output", "region", "status", "visits")
              .withColumn("finished_at_clean", date_format(col("finished_at"), "yyyy-MM-dd_HH-mm-ss"))
              .withColumn("dispatched_date", to_date(col("finished_at")))  # Crear columna para particionar por fecha de despacho
              .persist(StorageLevel.MEMORY_AND_DISK)  # Persistir en memoria para evitar recalcular
)


In [ ]:
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

# Escribir los datos transformados en la capa 'silver' particionando por dispatched_date
(silver_df
    .coalesce(10)  # Reducir el número de particiones durante la escritura para evitar muchas tareas simultáneas
    .write
    .mode('ignore')
    .partitionBy("dispatched_date")  # Particionar por fecha de despacho
    .parquet(silver_parquet_path)  # Guardar los datos en formato Parquet
)
# Unpersist the DataFrame after writing
silver_df.unpersist()

In [8]:
from pyspark.sql.functions import col

# Load the silver_df from the saved parquet files in the silver layer
silver_df = spark.read.parquet(silver_parquet_path)

# Now apply the optimized transformation to create partitioned_df
partitioned_df = silver_df \
    .select("finished_at_clean", col("output.polylines").alias("polylines")) \
    .repartition("finished_at_clean")  # Repartition by finished_at_clean to process in parallel

# Define the S3 path for the routes files
routes_file_path = "s3a://dispatched-orders/optimized-dispatched-gold/routes/"

# Write out the data partitioned by 'finished_at_clean'
partitioned_df.write \
    .mode("overwrite") \
    .partitionBy("finished_at_clean") \
    .parquet(routes_file_path)


24/10/05 21:26:40 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Grafos de recorridos codificados Streamlit (no PowerBI)

In [9]:
pBI_parquet_path = "s3a://dispatched-orders/optimized-dispatched-gold/pBI/"

# Load the Silver table from S3
silver_df = spark.read.parquet(silver_parquet_path)

# Drop 'pl_precision' and 'polylines' from the 'output' struct
clean_gold_df = (silver_df
                 .withColumn("output", 
                             col("output").dropFields("pl_precision", "polylines"))  # Dropping the fields
                 .select("output", "dispatched_date", "finished_at")  # Select only the required columns
                )

# Write the cleaned data to S3, partitioned by 'finished_at'
clean_gold_df.write.mode('overwrite').partitionBy("finished_at").parquet(pBI_parquet_path)

In [10]:
# Define the S3 path for Athena to store the query result
tables_location = "s3://dispatched-orders/optimized-dispatched-gold/pBI/tables/"  # Ensure this is the correct format

# Define the S3 location where the Parquet files are stored
pBI_file_path = "s3://dispatched-orders/optimized-dispatched-gold/pBI/"

In [11]:
# Define the query to create the database
create_database_query = """
CREATE DATABASE IF NOT EXISTS optimization_db
"""

# Execute the query to create the database
response = athena_client.start_query_execution(
    QueryString=create_database_query,
    ResultConfiguration={
        'OutputLocation': tables_location  # Output location for query results
    }
)

# Wait for the query execution to complete
query_execution_id = response['QueryExecutionId']
status = 'RUNNING'

# Wait for query completion
while status == 'RUNNING':
    response = athena_client.get_query_execution(QueryExecutionId=query_execution_id)
    status = response['QueryExecution']['Status']['State']
    print(f"Query status: {status}")
    if status == 'FAILED' or status == 'CANCELLED':
        raise Exception(f"Query {query_execution_id} failed or was cancelled")
    time.sleep(2)

print(f"Query {query_execution_id} succeeded!")

Query status: RUNNING
Query status: SUCCEEDED
Query 71fbfd61-856c-4e9e-9c77-883709fe3864 succeeded!


In [12]:
# SQL query to create the table in Athena
create_table_query = """
CREATE EXTERNAL TABLE IF NOT EXISTS optimization_db.dispatched_orders (
  output STRUCT<
    distances: STRUCT<
      driver_1: DOUBLE,
      driver_2: DOUBLE,
      driver_3: DOUBLE,
      driver_4: DOUBLE,
      driver_5: DOUBLE,
      driver_6: DOUBLE,
      driver_7: DOUBLE,
      driver_8: DOUBLE,
      driver_9: DOUBLE,
      driver_10: DOUBLE,
      driver_11: DOUBLE,
      driver_12: DOUBLE,
      driver_13: DOUBLE,
      driver_14: DOUBLE,
      driver_15: DOUBLE,
      driver_16: DOUBLE,
      driver_17: DOUBLE,
      driver_18: DOUBLE
    >,
    num_late_visits: BIGINT,
    num_unserved: BIGINT,
    solution: STRUCT<
      driver_1: ARRAY<STRUCT<
        arrival_time: STRING,
        distance: DOUBLE,
        finish_time: STRING,
        location_id: STRING,
        location_name: STRING
      >>,
      driver_2: ARRAY<STRUCT<
        arrival_time: STRING,
        distance: DOUBLE,
        finish_time: STRING,
        location_id: STRING,
        location_name: STRING
      >>,
      driver_3: ARRAY<STRUCT<
        arrival_time: STRING,
        distance: DOUBLE,
        finish_time: STRING,
        location_id: STRING,
        location_name: STRING
      >>,
      driver_4: ARRAY<STRUCT<
        arrival_time: STRING,
        distance: DOUBLE,
        finish_time: STRING,
        location_id: STRING,
        location_name: STRING
      >>,
      driver_5: ARRAY<STRUCT<
        arrival_time: STRING,
        distance: DOUBLE,
        finish_time: STRING,
        location_id: STRING,
        location_name: STRING
      >>,
      driver_6: ARRAY<STRUCT<
        arrival_time: STRING,
        distance: DOUBLE,
        finish_time: STRING,
        location_id: STRING,
        location_name: STRING
      >>,
      driver_7: ARRAY<STRUCT<
        arrival_time: STRING,
        distance: DOUBLE,
        finish_time: STRING,
        location_id: STRING,
        location_name: STRING
      >>,
      driver_8: ARRAY<STRUCT<
        arrival_time: STRING,
        distance: DOUBLE,
        finish_time: STRING,
        location_id: STRING,
        location_name: STRING
      >>,
      driver_9: ARRAY<STRUCT<
        arrival_time: STRING,
        distance: DOUBLE,
        finish_time: STRING,
        location_id: STRING,
        location_name: STRING
      >>,
      driver_10: ARRAY<STRUCT<
        arrival_time: STRING,
        distance: DOUBLE,
        finish_time: STRING,
        location_id: STRING,
        location_name: STRING
      >>,
      driver_11: ARRAY<STRUCT<
        arrival_time: STRING,
        distance: DOUBLE,
        finish_time: STRING,
        location_id: STRING,
        location_name: STRING
      >>,
      driver_12: ARRAY<STRUCT<
        arrival_time: STRING,
        distance: DOUBLE,
        finish_time: STRING,
        location_id: STRING,
        location_name: STRING
      >>,
      driver_13: ARRAY<STRUCT<
        arrival_time: STRING,
        distance: DOUBLE,
        finish_time: STRING,
        location_id: STRING,
        location_name: STRING
      >>,
      driver_14: ARRAY<STRUCT<
        arrival_time: STRING,
        distance: DOUBLE,
        finish_time: STRING,
        location_id: STRING,
        location_name: STRING
      >>,
      driver_15: ARRAY<STRUCT<
        arrival_time: STRING,
        distance: DOUBLE,
        finish_time: STRING,
        location_id: STRING,
        location_name: STRING
      >>,
      driver_16: ARRAY<STRUCT<
        arrival_time: STRING,
        distance: DOUBLE,
        finish_time: STRING,
        location_id: STRING,
        location_name: STRING
      >>,
      driver_17: ARRAY<STRUCT<
        arrival_time: STRING,
        distance: DOUBLE,
        finish_time: STRING,
        location_id: STRING,
        location_name: STRING
      >>,
      driver_18: ARRAY<STRUCT<
        arrival_time: STRING,
        distance: DOUBLE,
        finish_time: STRING,
        location_id: STRING,
        location_name: STRING
      >>
    >,
    status: STRING,
    total_break_time: BIGINT,
    total_distance: DOUBLE,
    total_idle_time: BIGINT,
    total_travel_time: BIGINT,
    total_vehicle_overtime: BIGINT,
    total_visit_lateness: BIGINT,
    total_working_time: BIGINT,
    unserved: STRING,
    vehicle_overtime: STRUCT<
      driver_1: BIGINT,
      driver_2: BIGINT,
      driver_3: BIGINT,
      driver_4: BIGINT,
      driver_5: BIGINT,
      driver_6: BIGINT,
      driver_7: BIGINT,
      driver_8: BIGINT,
      driver_9: BIGINT,
      driver_10: BIGINT,
      driver_11: BIGINT,
      driver_12: BIGINT,
      driver_13: BIGINT,
      driver_14: BIGINT,
      driver_15: BIGINT,
      driver_16: BIGINT,
      driver_17: BIGINT,
      driver_18: BIGINT
    >
  >,
  dispatched_date DATE
)
PARTITIONED BY (finished_at STRING)
STORED AS PARQUET
LOCATION 's3://dispatched-orders/optimized-dispatched-gold/pBI/'
TBLPROPERTIES ('parquet.compress'='SNAPPY');
"""



In [13]:
# Python code to run MSCK REPAIR TABLE
repair_table_query = "MSCK REPAIR TABLE optimization_db.dispatched_orders;"

# Execute the Athena query to create the table
response = athena_client.start_query_execution(
    QueryString=create_table_query,
    QueryExecutionContext={
        'Database': 'optimization_db'
    },
    ResultConfiguration={
        'OutputLocation': tables_location
    }
)

# Wait for the query to complete
query_execution_id = response['QueryExecutionId']
status = 'RUNNING'
while status == 'RUNNING':
    response = athena_client.get_query_execution(QueryExecutionId=query_execution_id)
    status = response['QueryExecution']['Status']['State']
    print(f"Query status: {status}")
    if status == 'FAILED' or status == 'CANCELLED':
        raise Exception(f"Query {query_execution_id} failed or was cancelled")
    time.sleep(2)

print(f"Query {query_execution_id} succeeded!")

Query status: RUNNING
Query status: SUCCEEDED
Query 86719d9b-2f5c-4793-b19f-643be4f37566 succeeded!


In [14]:
# Run MSCK REPAIR TABLE to load partitions
repair_table_query = "MSCK REPAIR TABLE optimization_db.dispatched_orders;"

response = athena_client.start_query_execution(
    QueryString=repair_table_query,
    QueryExecutionContext={
        'Database': 'optimization_db'
    },
    ResultConfiguration={
        'OutputLocation': tables_location
    }
)

# Wait for the query execution to complete
query_execution_id = response['QueryExecutionId']
status = 'RUNNING'

while status in ['RUNNING', 'QUEUED']:
    response = athena_client.get_query_execution(QueryExecutionId=query_execution_id)
    status = response['QueryExecution']['Status']['State']
    if status in ['FAILED', 'CANCELLED']:
        reason = response['QueryExecution']['Status']['StateChangeReason']
        raise Exception(f"MSCK REPAIR TABLE query {query_execution_id} failed or was cancelled. Reason: {reason}")
    print(f"MSCK REPAIR TABLE status: {status}")
    time.sleep(2)

print(f"MSCK REPAIR TABLE {query_execution_id} succeeded!")


MSCK REPAIR TABLE status: QUEUED
MSCK REPAIR TABLE status: RUNNING
MSCK REPAIR TABLE status: SUCCEEDED
MSCK REPAIR TABLE 40502f1b-39d8-47fd-b1c8-ad5c2fe01291 succeeded!


In [40]:
# Define the SELECT query
select_query = "SELECT * FROM optimization_db.dispatched_orders WHERE finished_at = '2024-10-05';"

# Execute the query
response = athena_client.start_query_execution(
    QueryString=select_query,
    QueryExecutionContext={
        'Database': 'optimization_db'
    },
    ResultConfiguration={
        'OutputLocation': tables_location
    }
)

# Wait for the query execution to complete
query_execution_id = response['QueryExecutionId']
status = 'RUNNING'

while status in ['RUNNING', 'QUEUED']:
    response = athena_client.get_query_execution(QueryExecutionId=query_execution_id)
    status = response['QueryExecution']['Status']['State']
    if status in ['FAILED', 'CANCELLED']:
        reason = response['QueryExecution']['Status']['StateChangeReason']
        raise Exception(f"SELECT query {query_execution_id} failed or was cancelled. Reason: {reason}")
    print(f"SELECT query status: {status}")
    time.sleep(2)

print(f"SELECT query {query_execution_id} succeeded!")

# Fetch the results
results_paginator = athena_client.get_paginator('get_query_results')
results_iterator = results_paginator.paginate(QueryExecutionId=query_execution_id)

for results_page in results_iterator:
    for row in results_page['ResultSet']['Rows']:
        print([col.get('VarCharValue', '') for col in row['Data']])


SELECT query status: QUEUED
SELECT query status: SUCCEEDED
SELECT query 1b52e54f-d277-4a76-acc3-5aa9a907231f succeeded!
['output', 'dispatched_date', 'finished_at']


In [20]:
# Define the first query to create dispatched_orders_flat view
create_dispatched_orders_flat_view = """
CREATE OR REPLACE VIEW optimization_db.dispatched_orders_flat AS
SELECT
  dispatched_date,
  finished_at,
  output.status,
  output.total_break_time,
  output.total_distance,
  output.total_idle_time,
  output.total_travel_time,
  output.total_vehicle_overtime,
  output.total_visit_lateness,
  output.total_working_time,
  output.num_late_visits,
  output.num_unserved,
  output.unserved,
  -- Flatten distances
  output.distances.driver_1 AS distance_driver_1,
  output.distances.driver_2 AS distance_driver_2,
  output.distances.driver_3 AS distance_driver_3,
  output.distances.driver_4 AS distance_driver_4,
  output.distances.driver_5 AS distance_driver_5,
  output.distances.driver_6 AS distance_driver_6,
  output.distances.driver_7 AS distance_driver_7,
  output.distances.driver_8 AS distance_driver_8,
  output.distances.driver_9 AS distance_driver_9,
  output.distances.driver_10 AS distance_driver_10,
  output.distances.driver_11 AS distance_driver_11,
  output.distances.driver_12 AS distance_driver_12,
  output.distances.driver_13 AS distance_driver_13,
  output.distances.driver_14 AS distance_driver_14,
  output.distances.driver_15 AS distance_driver_15,
  output.distances.driver_16 AS distance_driver_16,
  output.distances.driver_17 AS distance_driver_17,
  output.distances.driver_18 AS distance_driver_18,
  -- Flatten vehicle_overtime
  output.vehicle_overtime.driver_1 AS overtime_driver_1,
  output.vehicle_overtime.driver_2 AS overtime_driver_2,
  output.vehicle_overtime.driver_3 AS overtime_driver_3,
  output.vehicle_overtime.driver_4 AS overtime_driver_4,
  output.vehicle_overtime.driver_5 AS overtime_driver_5,
  output.vehicle_overtime.driver_6 AS overtime_driver_6,
  output.vehicle_overtime.driver_7 AS overtime_driver_7,
  output.vehicle_overtime.driver_8 AS overtime_driver_8,
  output.vehicle_overtime.driver_9 AS overtime_driver_9,
  output.vehicle_overtime.driver_10 AS overtime_driver_10,
  output.vehicle_overtime.driver_11 AS overtime_driver_11,
  output.vehicle_overtime.driver_12 AS overtime_driver_12,
  output.vehicle_overtime.driver_13 AS overtime_driver_13,
  output.vehicle_overtime.driver_14 AS overtime_driver_14,
  output.vehicle_overtime.driver_15 AS overtime_driver_15,
  output.vehicle_overtime.driver_16 AS overtime_driver_16,
  output.vehicle_overtime.driver_17 AS overtime_driver_17,
  output.vehicle_overtime.driver_18 AS overtime_driver_18
FROM optimization_db.dispatched_orders;
"""

# Function to execute Athena queries
def execute_athena_query(query, database, output_location):
    response = athena_client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={
            'Database': database
        },
        ResultConfiguration={
            'OutputLocation': output_location
        }
    )

    # Wait for query execution to complete
    query_execution_id = response['QueryExecutionId']
    status = 'RUNNING'

    while status in ['RUNNING', 'QUEUED']:
        response = athena_client.get_query_execution(QueryExecutionId=query_execution_id)
        status = response['QueryExecution']['Status']['State']
        if status in ['FAILED', 'CANCELLED']:
            reason = response['QueryExecution']['Status'].get('StateChangeReason', 'Unknown error')
            raise Exception(f"Query {query_execution_id} failed or was cancelled. Reason: {reason}")
        print(f"Query status: {status}")
        time.sleep(2)

    print(f"Query {query_execution_id} succeeded!")

# Execute the query to create dispatched_orders_flat view
print("Creating dispatched_orders_flat view...")
execute_athena_query(create_dispatched_orders_flat_view, 'optimization_db', tables_location)

Creating dispatched_orders_flat view...
Query status: RUNNING
Query status: SUCCEEDED
Query d950f853-7bf7-4d6f-81be-3cff190cfb06 succeeded!


In [21]:
# Build the corrected query for driver_solutions view
driver_solutions_query = """
CREATE OR REPLACE VIEW optimization_db.driver_solutions AS
"""

# Generate SELECT statements for each driver
for i in range(1, 19):
    driver_id = f'driver_{i}'
    select_statement = f"""
SELECT
  dispatched_date,
  finished_at,
  '{driver_id}' AS driver_id,
  t.elem.arrival_time,
  t.elem.distance,
  t.elem.finish_time,
  t.elem.location_id,
  t.elem.location_name
FROM optimization_db.dispatched_orders
CROSS JOIN UNNEST(output.solution.{driver_id}) AS t (elem)
"""
    if i > 1:
        driver_solutions_query += "\nUNION ALL\n"
    driver_solutions_query += select_statement

# Function to execute Athena queries
def execute_athena_query(query, database, output_location):
    response = athena_client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={
            'Database': database
        },
        ResultConfiguration={
            'OutputLocation': output_location
        }
    )

    # Wait for query execution to complete
    query_execution_id = response['QueryExecutionId']
    status = 'RUNNING'

    while status in ['RUNNING', 'QUEUED']:
        response = athena_client.get_query_execution(QueryExecutionId=query_execution_id)
        status = response['QueryExecution']['Status']['State']
        if status in ['FAILED', 'CANCELLED']:
            reason = response['QueryExecution']['Status'].get('StateChangeReason', 'Unknown error')
            raise Exception(f"Query {query_execution_id} failed or was cancelled. Reason: {reason}")
        print(f"Query status: {status}")
        time.sleep(2)

    print(f"Query {query_execution_id} succeeded!")

# Execute the query to create driver_solutions view
print("Creating driver_solutions view...")
execute_athena_query(driver_solutions_query, 'optimization_db', tables_location)

Creating driver_solutions view...
Query status: RUNNING
Query status: SUCCEEDED
Query 394f1e2d-41a4-4d33-914f-5db94818e8f1 succeeded!


In [22]:
def run_query(query, database, output_location):
    response = athena_client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={
            'Database': database
        },
        ResultConfiguration={
            'OutputLocation': output_location
        }
    )

    query_execution_id = response['QueryExecutionId']
    status = 'RUNNING'

    while status in ['RUNNING', 'QUEUED']:
        response = athena_client.get_query_execution(QueryExecutionId=query_execution_id)
        status = response['QueryExecution']['Status']['State']
        time.sleep(1)

    if status == 'SUCCEEDED':
        print(f"Query succeeded: {query_execution_id}")
        return query_execution_id
    else:
        reason = response['QueryExecution']['Status'].get('StateChangeReason', 'Unknown error')
        raise Exception(f"Query failed: {reason}")

Data for a Specific Driver

In [27]:
# Example usage:
query = """
SELECT *
FROM optimization_db.driver_solutions
WHERE driver_id = 'driver_18'
ORDER BY finished_at, arrival_time
LIMIT 20;
"""

try:
    query_execution_id = run_query(query, 'optimization_db', tables_location)
    # Fetch and display results
    results_paginator = athena_client.get_paginator('get_query_results')
    results_iterator = results_paginator.paginate(QueryExecutionId=query_execution_id)

    for results_page in results_iterator:
        for row in results_page['ResultSet']['Rows']:
            print([col.get('VarCharValue', '') for col in row['Data']])
except Exception as e:
    print(str(e))

Query succeeded: 9599d6a4-e04d-4e9b-bf9a-f04948c8e6d1
['dispatched_date', 'finished_at', 'driver_id', 'arrival_time', 'distance', 'finish_time', 'location_id', 'location_name']
['2024-10-02', '2024-10-02T22:59:31.256Z', 'driver_18', '09:00', '0.0', '', 'depot_3', 'Vallecas']
['2024-10-02', '2024-10-02T22:59:31.256Z', 'driver_18', '09:00', '0.0', '', 'depot_3', 'Vallecas']
['2024-10-02', '2024-10-02T23:50:50.402Z', 'driver_18', '09:00', '0.0', '', 'depot_3', 'Vallecas']
['2024-10-02', '2024-10-02T23:50:50.402Z', 'driver_18', '09:43', '33387.6', '09:48', '5c75b320-fa00-41ee-a777-cacd94700480', 'cus-1545ae0d-2867-4ccd-9295-7b5e81593bf5']
['2024-10-02', '2024-10-02T23:50:50.402Z', 'driver_18', '10:03', '5830.8', '10:08', '726dce59-8955-48a0-87ed-1d8bed022840', 'cus-dd367be8-f54f-42aa-bb3a-b3ea00b7ea85']
['2024-10-02', '2024-10-02T23:50:50.402Z', 'driver_18', '10:15', '1616.9', '10:20', 'cf988e65-f487-446d-a219-3042b52217a7', 'cus-44773914-37e8-46e8-82ad-0f6d41a2d029']
['2024-10-02', '2024-

Joining the Views to Combine Data

In [28]:
query = """
SELECT
  ds.finished_at,
  ds.dispatched_date,
  ds.driver_id,
  ds.arrival_time,
  ds.distance AS segment_distance,
  ds.finish_time,
  ds.location_id,
  ds.location_name,
  dof.status,
  dof.total_distance,
  dof.total_travel_time,
  -- Include driver's total distance from the flat view
  CASE ds.driver_id
    WHEN 'driver_1' THEN dof.distance_driver_1
    WHEN 'driver_2' THEN dof.distance_driver_2
    WHEN 'driver_3' THEN dof.distance_driver_3
    WHEN 'driver_4' THEN dof.distance_driver_4
    WHEN 'driver_5' THEN dof.distance_driver_5
    WHEN 'driver_6' THEN dof.distance_driver_6
    WHEN 'driver_7' THEN dof.distance_driver_7
    WHEN 'driver_8' THEN dof.distance_driver_8
    WHEN 'driver_9' THEN dof.distance_driver_9
    WHEN 'driver_10' THEN dof.distance_driver_10
    WHEN 'driver_11' THEN dof.distance_driver_11
    WHEN 'driver_12' THEN dof.distance_driver_12
    WHEN 'driver_13' THEN dof.distance_driver_13
    WHEN 'driver_14' THEN dof.distance_driver_14
    WHEN 'driver_15' THEN dof.distance_driver_15
    WHEN 'driver_16' THEN dof.distance_driver_16
    WHEN 'driver_17' THEN dof.distance_driver_17
    WHEN 'driver_18' THEN dof.distance_driver_18
    ELSE NULL
  END AS total_driver_distance
FROM optimization_db.driver_solutions ds
JOIN optimization_db.dispatched_orders_flat dof
  ON ds.finished_at = dof.finished_at
ORDER BY ds.finished_at, ds.driver_id, ds.arrival_time
;
"""

try:
    query_execution_id = run_query(query, 'optimization_db', tables_location)
    # Fetch and display results
    results_paginator = athena_client.get_paginator('get_query_results')
    results_iterator = results_paginator.paginate(QueryExecutionId=query_execution_id)

    for results_page in results_iterator:
        for row in results_page['ResultSet']['Rows']:
            print([col.get('VarCharValue', '') for col in row['Data']])
except Exception as e:
    print(str(e))

Query succeeded: e3cda17d-1436-4df2-9bcb-756882c10801
['finished_at', 'dispatched_date', 'driver_id', 'arrival_time', 'segment_distance', 'finish_time', 'location_id', 'location_name', 'status', 'total_distance', 'total_travel_time', 'total_driver_distance']
['2024-10-02T22:59:31.256Z', '2024-10-02', 'driver_1', '09:00', '0.0', '', 'depot_3', 'Vallecas', 'success', '521.5519', '1058', '28.0121']
['2024-10-02T22:59:31.256Z', '2024-10-02', 'driver_1', '09:34', '13411.1', '09:39', '2b9739b1-9711-465a-bcda-87f0fea426e2', 'cus-a83c1637-ffed-41a2-aede-43d0cf122fe3', 'success', '521.5519', '1058', '28.0121']
['2024-10-02T22:59:31.256Z', '2024-10-02', 'driver_1', '10:09', '14601.0', '', 'depot_3', 'Vallecas', 'success', '521.5519', '1058', '28.0121']
['2024-10-02T22:59:31.256Z', '2024-10-02', 'driver_10', '09:00', '0.0', '', 'depot_3', 'Vallecas', 'success', '521.5519', '1058', '0.0']
['2024-10-02T22:59:31.256Z', '2024-10-02', 'driver_10', '09:00', '0.0', '', 'depot_3', 'Vallecas', 'success', 

Aggregating Driver Data

In [29]:
query = """
SELECT
  driver_id,
  SUM(distance) AS total_route_distance,
  COUNT(*) AS num_stops
FROM optimization_db.driver_solutions
GROUP BY driver_id
ORDER BY driver_id;
"""

try:
    query_execution_id = run_query(query, 'optimization_db', tables_location)
    # Fetch and display results
    results_paginator = athena_client.get_paginator('get_query_results')
    results_iterator = results_paginator.paginate(QueryExecutionId=query_execution_id)

    for results_page in results_iterator:
        for row in results_page['ResultSet']['Rows']:
            print([col.get('VarCharValue', '') for col in row['Data']])
except Exception as e:
    print(str(e))

Query succeeded: d7b9572f-42d2-4120-99f3-815affd64052
['driver_id', 'total_route_distance', 'num_stops']
['driver_1', '567949.9', '61']
['driver_10', '632123.8', '68']
['driver_11', '658336.2000000001', '70']
['driver_12', '710258.5999999999', '66']
['driver_13', '587028.7000000001', '66']
['driver_14', '459763.0', '59']
['driver_15', '477458.69999999995', '54']
['driver_16', '341869.1', '55']
['driver_17', '471847.69999999995', '61']
['driver_18', '469428.60000000003', '52']
['driver_2', '587912.8', '61']
['driver_3', '642653.3', '68']
['driver_4', '691738.9999999999', '60']
['driver_5', '589574.1', '67']
['driver_6', '728770.2000000001', '58']
['driver_7', '740515.0', '73']
['driver_8', '735700.5', '60']
['driver_9', '732241.5999999999', '58']


Driver Performance Metrics

In [30]:
query = """
SELECT
  driver_id,
  dispatched_date,
  finished_at,
  CASE driver_id
    WHEN 'driver_1' THEN distance_driver_1
    WHEN 'driver_2' THEN distance_driver_2
    WHEN 'driver_3' THEN distance_driver_3
    WHEN 'driver_4' THEN distance_driver_4
    WHEN 'driver_5' THEN distance_driver_5
    WHEN 'driver_6' THEN distance_driver_6
    WHEN 'driver_7' THEN distance_driver_7
    WHEN 'driver_8' THEN distance_driver_8
    WHEN 'driver_9' THEN distance_driver_9
    WHEN 'driver_10' THEN distance_driver_10
    WHEN 'driver_11' THEN distance_driver_11
    WHEN 'driver_12' THEN distance_driver_12
    WHEN 'driver_13' THEN distance_driver_13
    WHEN 'driver_14' THEN distance_driver_14
    WHEN 'driver_15' THEN distance_driver_15
    WHEN 'driver_16' THEN distance_driver_16
    WHEN 'driver_17' THEN distance_driver_17
    WHEN 'driver_18' THEN distance_driver_18
    ELSE NULL
  END AS total_distance,
  CASE driver_id
    WHEN 'driver_1' THEN overtime_driver_1
    WHEN 'driver_2' THEN overtime_driver_2
    WHEN 'driver_3' THEN overtime_driver_3
    WHEN 'driver_4' THEN overtime_driver_4
    WHEN 'driver_5' THEN overtime_driver_5
    WHEN 'driver_6' THEN overtime_driver_6
    WHEN 'driver_7' THEN overtime_driver_7
    WHEN 'driver_8' THEN overtime_driver_8
    WHEN 'driver_9' THEN overtime_driver_9
    WHEN 'driver_10' THEN overtime_driver_10
    WHEN 'driver_11' THEN overtime_driver_11
    WHEN 'driver_12' THEN overtime_driver_12
    WHEN 'driver_13' THEN overtime_driver_13
    WHEN 'driver_14' THEN overtime_driver_14
    WHEN 'driver_15' THEN overtime_driver_15
    WHEN 'driver_16' THEN overtime_driver_16
    WHEN 'driver_17' THEN overtime_driver_17
    WHEN 'driver_18' THEN overtime_driver_18
    ELSE NULL
  END AS total_overtime
FROM optimization_db.dispatched_orders_flat,
UNNEST(
  ARRAY['driver_1', 'driver_2', 'driver_3', 'driver_4', 'driver_5', 'driver_6', 'driver_7', 'driver_8', 'driver_9', 'driver_10',
        'driver_11', 'driver_12', 'driver_13', 'driver_14', 'driver_15', 'driver_16', 'driver_17', 'driver_18']
) AS t (driver_id)
ORDER BY driver_id, dispatched_date;
"""

try:
    query_execution_id = run_query(query, 'optimization_db', tables_location)
    # Fetch and display results
    results_paginator = athena_client.get_paginator('get_query_results')
    results_iterator = results_paginator.paginate(QueryExecutionId=query_execution_id)

    for results_page in results_iterator:
        for row in results_page['ResultSet']['Rows']:
            print([col.get('VarCharValue', '') for col in row['Data']])
except Exception as e:
    print(str(e))



Query succeeded: 2106f22a-fd2e-4264-8105-ccd47f2ba98e
['driver_id', 'dispatched_date', 'finished_at', 'total_distance', 'total_overtime']
['driver_1', '2024-10-02', '2024-10-02T23:50:50.402Z', '41.7215', '0']
['driver_1', '2024-10-02', '2024-10-02T22:59:31.256Z', '28.0121', '0']
['driver_1', '2024-10-03', '2024-10-03T01:31:08.949Z', '40.2775', '0']
['driver_1', '2024-10-05', '2024-10-05T18:15:50.586Z', '86.2955', '0']
['driver_1', '2024-10-05', '2024-10-05T15:41:44.709Z', '31.7539', '0']
['driver_1', '2024-10-05', '2024-10-05T03:39:43.902Z', '90.6959', '0']
['driver_1', '2024-10-05', '2024-10-05T15:39:39.287Z', '31.7539', '0']
['driver_1', '2024-10-05', '2024-10-05T15:36:20.257Z', '48.831300000000006', '0']
['driver_1', '2024-10-05', '2024-10-05T11:18:15.630Z', '69.1904', '']
['driver_1', '2024-10-05', '2024-10-05T18:12:07.953Z', '99.41789999999999', '0']
['driver_10', '2024-10-02', '2024-10-02T22:59:31.256Z', '0.0', '0']
['driver_10', '2024-10-02', '2024-10-02T23:50:50.402Z', '75.7179

Analyzing Unserved Orders

In [31]:
query = """
SELECT
  dispatched_date,
  finished_at,
  unserved
FROM optimization_db.dispatched_orders_flat
WHERE unserved IS NOT NULL AND unserved <> '';
"""
try:
    query_execution_id = run_query(query, 'optimization_db', tables_location)
    # Fetch and display results
    results_paginator = athena_client.get_paginator('get_query_results')
    results_iterator = results_paginator.paginate(QueryExecutionId=query_execution_id)

    for results_page in results_iterator:
        for row in results_page['ResultSet']['Rows']:
            print([col.get('VarCharValue', '') for col in row['Data']])
except Exception as e:
    print(str(e))



Query failed: HIVE_CANNOT_OPEN_SPLIT: Error opening Hive split s3://dispatched-orders/optimized-dispatched-gold/pBI/finished_at=2024-10-05T15%3A36%3A20.257Z/part-00007-8a8046ae-afd3-4875-b92e-1e4a5cf4ac83.c000.snappy.parquet (offset=0, length=312651): class org.apache.parquet.io.GroupColumnIO cannot be cast to class org.apache.parquet.io.PrimitiveColumnIO (org.apache.parquet.io.GroupColumnIO and org.apache.parquet.io.PrimitiveColumnIO are in unnamed module of loader io.trino.server.PluginClassLoader @79becbd4)


Orders with Late Visits

In [32]:
query = """
SELECT
  dispatched_date,
  finished_at,
  num_late_visits,
  total_visit_lateness
FROM optimization_db.dispatched_orders_flat
WHERE num_late_visits > 0;
"""
try:
    query_execution_id = run_query(query, 'optimization_db', tables_location)
    # Fetch and display results
    results_paginator = athena_client.get_paginator('get_query_results')
    results_iterator = results_paginator.paginate(QueryExecutionId=query_execution_id)

    for results_page in results_iterator:
        for row in results_page['ResultSet']['Rows']:
            print([col.get('VarCharValue', '') for col in row['Data']])
except Exception as e:
    print(str(e))



Query succeeded: 647a446a-3093-4e95-9859-e956a0baba7b
['dispatched_date', 'finished_at', 'num_late_visits', 'total_visit_lateness']


In [33]:
query = """
SELECT
  ds.driver_id,
  ds.dispatched_date,
  ds.finished_at,
  ds.arrival_time,
  ds.finish_time,
  ds.distance AS segment_distance,
  ds.location_id,
  ds.location_name
FROM optimization_db.driver_solutions ds
ORDER BY ds.driver_id, ds.finished_at, ds.arrival_time;
"""
try:
    query_execution_id = run_query(query, 'optimization_db', tables_location)
    # Fetch and display results
    results_paginator = athena_client.get_paginator('get_query_results')
    results_iterator = results_paginator.paginate(QueryExecutionId=query_execution_id)

    for results_page in results_iterator:
        for row in results_page['ResultSet']['Rows']:
            print([col.get('VarCharValue', '') for col in row['Data']])
except Exception as e:
    print(str(e))

Query succeeded: 8dd3f81b-765f-4ea3-a174-434807632c18
['driver_id', 'dispatched_date', 'finished_at', 'arrival_time', 'finish_time', 'segment_distance', 'location_id', 'location_name']
['driver_1', '2024-10-02', '2024-10-02T22:59:31.256Z', '09:00', '', '0.0', 'depot_3', 'Vallecas']
['driver_1', '2024-10-02', '2024-10-02T22:59:31.256Z', '09:34', '09:39', '13411.1', '2b9739b1-9711-465a-bcda-87f0fea426e2', 'cus-a83c1637-ffed-41a2-aede-43d0cf122fe3']
['driver_1', '2024-10-02', '2024-10-02T22:59:31.256Z', '10:09', '', '14601.0', 'depot_3', 'Vallecas']
['driver_1', '2024-10-02', '2024-10-02T23:50:50.402Z', '09:00', '', '0.0', 'depot_3', 'Vallecas']
['driver_1', '2024-10-02', '2024-10-02T23:50:50.402Z', '09:36', '09:41', '18676.0', '04da7395-06f5-41fd-956e-36012baa5e1d', 'cus-abfc2e74-4bb3-45a0-9161-adc591737c0d']
['driver_1', '2024-10-02', '2024-10-02T23:50:50.402Z', '09:50', '09:55', '1944.1', '655a5164-a06e-4e6c-8435-8099103aef47', 'cus-bcb273b4-4b6b-486e-b514-a76803a5bb96']
['driver_1', '

Data from the driver_solutions View

In [35]:
# Example usage:
query = """
SELECT *
FROM optimization_db.driver_solutions;
"""

try:
    query_execution_id = run_query(query, 'optimization_db', tables_location)
    # Fetch and display results
    results_paginator = athena_client.get_paginator('get_query_results')
    results_iterator = results_paginator.paginate(QueryExecutionId=query_execution_id)

    for results_page in results_iterator:
        for row in results_page['ResultSet']['Rows']:
            print([col.get('VarCharValue', '') for col in row['Data']])
except Exception as e:
    print(str(e))

Query succeeded: 087741b3-d5ce-41cd-beff-43b50b2635ce
['dispatched_date', 'finished_at', 'driver_id', 'arrival_time', 'distance', 'finish_time', 'location_id', 'location_name']
['2024-10-05', '2024-10-05T03:39:43.902Z', 'driver_3', '09:00', '0.0', '', 'depot_3', 'Vallecas']
['2024-10-05', '2024-10-05T03:39:43.902Z', 'driver_3', '09:18', '5563.2', '09:23', '4a8002ca-c5be-41cb-b7ba-f65660d2cc76', 'cus-e7ae6bd4-3afe-44c9-b301-c651058f0374']
['2024-10-05', '2024-10-05T03:39:43.902Z', 'driver_3', '09:34', '2200.2', '09:39', 'fe3fdc67-3c9a-4b26-b7a1-4178391fd522', 'cus-261ce26b-1b90-4fe2-9960-291fe92732d9']
['2024-10-05', '2024-10-05T03:39:43.902Z', 'driver_3', '10:02', '6593.4', '10:07', 'c737a2a4-accf-47b3-b0ca-bd400aad270e', 'cus-c587667d-ca80-4cc3-8572-f16b625649ad']
['2024-10-05', '2024-10-05T03:39:43.902Z', 'driver_3', '10:25', '5791.2', '10:30', 'f4c1970f-841a-4a7b-9278-58bb590c792c', 'cus-68360b1c-e5eb-4263-8fcd-619cd3d6ec8d']
['2024-10-05', '2024-10-05T03:39:43.902Z', 'driver_3', '1